# Week 21: MLOps - Model Deployment & Monitoring

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Model Versioning**: Tracking experiments and models
- **Deployment Patterns**: Batch, real-time, edge
- **Monitoring**: Drift detection, performance tracking
- **CI/CD for ML**: Automated pipelines

---

## Why MLOps?

Research → Production gap:
- 87% of ML projects never reach production
- Models degrade over time
- Reproducibility is critical

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import hashlib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 21: MLOps - Deployment & Monitoring")

---

## Part 1: Model Versioning

### Key Concepts

- **Experiment tracking**: Log parameters, metrics, artifacts
- **Model registry**: Store and version models
- **Data versioning**: Track training data

### Tools

- MLflow, Weights & Biases, DVC

In [ ]:
class SimpleExperimentTracker:
    """Basic experiment tracking (conceptual MLflow-like)"""
    
    def __init__(self):
        self.experiments = []
        self.current_run = None
    
    def start_run(self, name):
        self.current_run = {
            'name': name,
            'run_id': hashlib.md5(f"{name}{datetime.now()}".encode()).hexdigest()[:8],
            'timestamp': datetime.now().isoformat(),
            'params': {},
            'metrics': {},
            'artifacts': []
        }
        return self
    
    def log_param(self, key, value):
        self.current_run['params'][key] = value
    
    def log_metric(self, key, value):
        self.current_run['metrics'][key] = value
    
    def end_run(self):
        self.experiments.append(self.current_run)
        run_id = self.current_run['run_id']
        self.current_run = None
        return run_id
    
    def get_best_run(self, metric, maximize=True):
        valid_runs = [e for e in self.experiments if metric in e['metrics']]
        if not valid_runs:
            return None
        key_func = lambda x: x['metrics'][metric]
        return max(valid_runs, key=key_func) if maximize else min(valid_runs, key=key_func)

# Usage example
tracker = SimpleExperimentTracker()

# Simulate multiple experiments
for lr in [0.001, 0.01, 0.1]:
    for depth in [3, 5, 7]:
        tracker.start_run(f"xgboost_lr{lr}_d{depth}")
        tracker.log_param('learning_rate', lr)
        tracker.log_param('max_depth', depth)
        
        # Simulated metrics
        sharpe = 1.2 + np.random.randn() * 0.3 - 0.1 * abs(np.log10(lr) + 2)
        tracker.log_metric('sharpe', sharpe)
        tracker.log_metric('max_drawdown', -np.random.uniform(0.05, 0.15))
        
        tracker.end_run()

# Find best model
best = tracker.get_best_run('sharpe', maximize=True)
print("Best Experiment:")
print(f"  Run: {best['name']}")
print(f"  Params: {best['params']}")
print(f"  Sharpe: {best['metrics']['sharpe']:.3f}")

---

## Part 2: Deployment Patterns

### Batch Prediction

- Run predictions on schedule (daily, hourly)
- Good for: Portfolio rebalancing, end-of-day signals

### Real-time Prediction

- REST API / gRPC serving
- Good for: HFT, immediate execution

### Streaming

- Process data as it arrives (Kafka, Kinesis)
- Good for: Continuous monitoring, alerts

In [ ]:
class ModelServingSimulator:
    """Simulate different serving patterns"""
    
    def __init__(self, model_name, latency_ms=10):
        self.model_name = model_name
        self.latency_ms = latency_ms
        self.prediction_count = 0
    
    def predict(self, features):
        """Real-time prediction"""
        import time
        time.sleep(self.latency_ms / 1000)  # Simulate latency
        self.prediction_count += 1
        # Dummy prediction
        return np.random.choice([-1, 0, 1], p=[0.3, 0.4, 0.3])
    
    def batch_predict(self, features_batch):
        """Batch prediction"""
        predictions = []
        for features in features_batch:
            predictions.append(self.predict(features))
        return predictions

# Architecture diagram (conceptual)
print("Typical ML Deployment Architecture")
print("="*60)
print("""
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   Market    │───▶│   Feature   │───▶│   Model     │
│   Data      │    │   Pipeline  │    │   Serving   │
└─────────────┘    └─────────────┘    └─────────────┘
                                            │
                   ┌─────────────┐          │
                   │   Order     │◀─────────┘
                   │   Manager   │
                   └─────────────┘
""")

---

## Part 3: Model Monitoring

### Types of Drift

1. **Data Drift**: Input distribution changes
2. **Concept Drift**: Relationship between X and Y changes
3. **Prediction Drift**: Model outputs change

### Detection Methods

- Statistical tests (KS, PSI)
- Moving average comparisons
- Distribution plots

In [ ]:
def population_stability_index(expected, actual, buckets=10):
    """Calculate PSI for drift detection"""
    # Create buckets based on expected distribution
    breakpoints = np.percentile(expected, np.linspace(0, 100, buckets + 1))
    breakpoints[0] = -np.inf
    breakpoints[-1] = np.inf
    
    expected_counts = np.histogram(expected, breakpoints)[0]
    actual_counts = np.histogram(actual, breakpoints)[0]
    
    # Avoid division by zero
    expected_pct = (expected_counts + 0.001) / len(expected)
    actual_pct = (actual_counts + 0.001) / len(actual)
    
    psi = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    return psi

# Simulate drift
n_samples = 1000

# Training distribution
train_feature = np.random.normal(0, 1, n_samples)

# Production distributions
prod_no_drift = np.random.normal(0, 1, n_samples)
prod_mild_drift = np.random.normal(0.3, 1.1, n_samples)
prod_severe_drift = np.random.normal(1.0, 1.5, n_samples)

print("PSI Drift Detection")
print("="*50)
print(f"No drift:     PSI = {population_stability_index(train_feature, prod_no_drift):.4f}")
print(f"Mild drift:   PSI = {population_stability_index(train_feature, prod_mild_drift):.4f}")
print(f"Severe drift: PSI = {population_stability_index(train_feature, prod_severe_drift):.4f}")
print("\nInterpretation: PSI < 0.1 (no drift), 0.1-0.2 (moderate), > 0.2 (significant)")

In [ ]:
# Visualize drift
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, (name, data) in zip(axes, [
    ('No Drift', prod_no_drift),
    ('Mild Drift', prod_mild_drift),
    ('Severe Drift', prod_severe_drift)
]):
    ax.hist(train_feature, bins=30, alpha=0.5, label='Training', density=True)
    ax.hist(data, bins=30, alpha=0.5, label='Production', density=True)
    psi = population_stability_index(train_feature, data)
    ax.set_title(f'{name}\nPSI = {psi:.4f}')
    ax.legend()
    ax.set_xlabel('Feature Value')

plt.tight_layout()
plt.show()

---

## Part 4: Performance Monitoring Dashboard

### Key Metrics to Track

| Metric | Purpose | Alert Threshold |
|--------|---------|----------------|
| Sharpe (rolling) | Performance | < 0.5 |
| Max Drawdown | Risk | > 10% |
| Hit Rate | Signal quality | < 45% |
| Feature PSI | Data drift | > 0.2 |

In [ ]:
class PerformanceMonitor:
    """Monitor model performance in production"""
    
    def __init__(self, sharpe_threshold=0.5, drawdown_threshold=0.10):
        self.sharpe_threshold = sharpe_threshold
        self.drawdown_threshold = drawdown_threshold
        self.returns = []
        self.alerts = []
    
    def add_return(self, ret):
        self.returns.append(ret)
        self._check_alerts()
    
    def _check_alerts(self):
        if len(self.returns) < 20:
            return
        
        recent = np.array(self.returns[-60:])  # Last 60 periods
        
        # Rolling Sharpe
        sharpe = recent.mean() / recent.std() * np.sqrt(252)
        if sharpe < self.sharpe_threshold:
            self.alerts.append(f"LOW SHARPE: {sharpe:.2f}")
        
        # Max Drawdown
        cum = (1 + recent).cumprod()
        peak = np.maximum.accumulate(cum)
        dd = (cum - peak) / peak
        max_dd = dd.min()
        if abs(max_dd) > self.drawdown_threshold:
            self.alerts.append(f"HIGH DRAWDOWN: {max_dd:.1%}")
    
    def get_status(self):
        if len(self.returns) < 20:
            return "INSUFFICIENT DATA"
        if self.alerts:
            return f"WARNING: {len(self.alerts)} alerts"
        return "HEALTHY"

# Simulate monitoring
monitor = PerformanceMonitor()

# Add daily returns
for _ in range(100):
    ret = np.random.normal(0.0003, 0.015)  # Slightly positive expected return
    monitor.add_return(ret)

print(f"Model Status: {monitor.get_status()}")
print(f"Alerts: {monitor.alerts[-3:] if monitor.alerts else 'None'}")

---

## Interview Questions

### Conceptual
1. What's the difference between data drift and concept drift?
2. Why do most ML projects fail to reach production?
3. How often should you retrain models?

### Technical
1. How do you detect model degradation?
2. What's your approach to A/B testing trading models?
3. How do you handle model rollback?

### Finance-Specific
1. What's unique about deploying trading models vs. other ML?
2. How do you handle regime changes?
3. What metrics do you monitor in production?

---

## Key Takeaways

| Component | Tool | Purpose |
|-----------|------|--------|
| Tracking | MLflow | Experiment management |
| Serving | FastAPI/Flask | Model inference |
| Monitoring | Grafana/Custom | Performance tracking |
| Pipeline | Airflow/Prefect | Orchestration |